# Download ffmped static build
Downloading a static build of FFmpeg to the Coursera server, unziping it, and adding the directory to the environment variable “PATH”.

In [1]:
# code copied from coursera lab
# Download latest FFmpeg static build and run ffmpeg locally
exist = !which ffmpeg
if not exist:
  !curl https://johnvansickle.com/ffmpeg/releases/ffmpeg-release-amd64-static.tar.xz -o ffmpeg.tar.xz \
     && tar -xf ffmpeg.tar.xz && rm ffmpeg.tar.xz
  ffmdir = !find . -iname ffmpeg-*-static
  path = %env PATH
  path = path + ':' + ffmdir[0]
  %env PATH $path

!which ffmpeg

/usr/bin/ffmpeg


# Function for examining the format of the films 

In [1]:
#import libary
import pathlib

In [2]:
class FFprobe():
    def __init__(self,filePath):
        self.filepath = filePath
        # set check variable to true until some specification isn't met
        self.isOkFormat = True
        # save video metadata in a list
        self.video_info = !ffprobe -v error -select_streams v:0 -show_entries stream=codec_name,r_frame_rate,display_aspect_ratio,width,height,bit_rate -of default=noprint_wrappers=1:nokey=1 $self.filepath
        # save audio metadata in a list
        self.audio_info = !ffprobe -v 0 -select_streams a:0 -show_entries stream=codec_name,channels,bit_rate -of default=noprint_wrappers=1:nokey=1 $self.filepath
    
    # function to determine which aspects of the digital file format 
    # does not meet the required specifications and print to txt.file which fields are problematic
    # and convert the incorrect formatted file to correct format automatically
    def checkFile(self):
        # heading for report include which file is being reported about
        report = f'Report for file {self. filepath}\n'
        # store the errors field
        error = ""
        
        # check if file format is correct
        # get the extension of the path without the file name
        file_extension = pathlib.Path(self. filepath).suffix
        if file_extension != '.mp4':
            self. isOkFormat = False
            error += f'-- Wrong video container format. Expected mp4, found {file_extension} \n'
        
        #check if the video codec is correct
        video_codec = self.video_info[0]
        if video_codec != 'h264':
            self. isOkFormat = False
            error += f'-- Wrong video codec. Expected h.264, found {video_codec}\n'

        #check if the frame rate is correct
        frame_rate = self.video_info[4]
        if frame_rate != "25/1":
            self. isOkFormat = False
            fps = frame_rate.split('/')
            fps = float(fps[0])/float(fps[1])
            error += f'-- Wrong frame rate. Expected 25 FPS, found {fps} FPS \n'

        #check if the aspect ratio is correct
        aspect_ratio = self.video_info[3]
        if aspect_ratio != '16:9':
            self. isOkFormat = False
            error += f'-- Wrong aspect ratio. Expected 16:9, found {aspect_ratio}\n'

        #check if the resolution is correct
        resolution = str(self.video_info[1])+"x"+str(self.video_info[2])
        if resolution != '640x360':
            self. isOkFormat = False
            error += f'-- Wrong resolution. Expected 640x360, found {resolution}\n'

        #check if the video bit rate is correct
        #convert video bit rate to Mb
        v_bit_rate = float(self.video_info[5])/1000000
        if v_bit_rate <= 2 or v_bit_rate >= 5  :
            self. isOkFormat = False
            error += f'-- Wrong video bit rate. Expected 2 - 5 Mb/s, found {v_bit_rate} Mb/s\n'
        
        #check if the audio bit rate is correct
        # convert audio bit rate to kb
        a_bit_rate = float(self.audio_info[2])/1000
        if a_bit_rate > 256:
            self. isOkFormat = False
            error += f'-- Wrong audio bit rate. Expected up to 256 kb/s, found {a_bit_rate} kb/s\n'

        #check if the audio codec is correct
        audio_codec = self.audio_info[0]
        if audio_codec != 'aac':
            self. isOkFormat = False
            error += f'-- Wrong audio codec. Expected aac, found {audio_codec}\n'

        #check if audio channels is correct
        channel_type =""
        # find the channel_type from the channels
        if (int(self.audio_info[1]) == 2):
            channel_type = 'stereo'
        elif (int(self.audio_info[1]) == 1):
            channel_type = 'mono'
        if channel_type != 'stereo':
            self. isOkFormat = False
            error += f'-- Wrong audio channels. Expected stereo, found {channel_type}\n'

        # if the file format is incorrect, update to correct file specification
        if self. isOkFormat == False:
            # get the file name of the path without the extension
            index = self.filepath.index('.')
            file_name = self.filepath[:index]
            # declare the output file name
            outputFile = file_name+'_formatOk.mp4'
            # covert the file format to correct specificaions
            !ffmpeg -loglevel error -hide_banner -nostats -y -i $self.filepath -c:v h264 -vf scale=640:360,setdar=16/9,fps=25 -c:a aac -ac 2 -b:v 3.5M -b:a 256k $outputFile
            # add the error fields text to the report
            report += error
            
            #Put the error report into txt file
            #open .txt file to write to
            with open('report.txt','a') as f:
                f.write(report)
                f.write('='*90)
                f.write('\n')
                f.close()  
        # if the file format is correct, dont do anything
        else:
            report += f'-- All formatting are okay'
        
        # print the report as output of the function
        return report



## FFprobe
ffprobe was used to gathers information from multimedia streams and put the information into the list. it can be used to check the format of the container used by a multimedia stream and the format and type of each media stream in it. 

To get the video metadata in a list, firstly "-v error" option is added to the 'ffprobe' command to reduce the information printed as an output where it  setting the log level to only display error messages. By mentioning "-select_streams v:0", we tell ffprobe to inspect the video stream with index=1. ffprobe has streams specifiers that allow the specific information to be extracted. Adding "show entries" option along with the key that we wish to extract to extract such as "stream=codec_name,r_frame_rate..", this tell ffprobe to only extract the key we specific in the stream. Adding "-of default=noprint_wrappers=1:nokey=1" to the command line, it set the output printing format where the wrapper and the key of each field will not be shown. The output of the ffprobe command line for the video metadata will be printed in the sequence of video codec,width,height,aspect ratio,frame rate and video bit rate.

To get the audio metadata, similar command will be used but "-v 0 -select_streams a:0" will be added to the command line instead. "-v 0" will not print any informational message,warnings or erros while "select_streams a:0" will inspect the audio stream with index=1. The output of the ffprobe command line for the audio metadata will be printed in the sequence of audio codec name, channels and audio bit rate. 

## FFmpeg
Ffmpeg is a fast video and audio converter that can grab from a live audio/video source.

After the format checking with if conditions, if any of the file format was found incorrect it will used the ffmpeg command line to correct the file to the required format. The command line include all the format of films that the organization specific, and it will be applied to the incorrect formatted file. Firstly, to convert the file to MP4 will be specific the output as mp4 format by adding the filename and "FormatOk" along with the mp4 extension at the back. "loglevel error -hide_banner -stats" show only the warning and nothing else. To influence the quality of each stream, "-c:v h264" will convert video to h264 codec, "-c:a aac" convert audio to aac codec, "-ac 2" convert audio channel to 2 which is stereo channel type, "-b:v" convert the video rate to 3.5M and "-b:a" convert the audio bit rate to 256k. To convert the resolution,aspect ratio and fps of the video, we used the video filter with the filtergraph expression where "-vf scale=640:360,setdar=16/9,fps=25" is added to the command line.



# Verify the application 
## Check the submitted file format

In [3]:
flims_name = ["Cosmos_War_of_the_Planets.mp4","Last_man_on_earth_1964.mov","The_Gun_and_the_Pulpit.avi",
    "The_Hill_Gang_Rides_Again.mp4","Voyage_to_the_Planet_of_Prehistoric_Women.mp4"]

In [4]:
file_loc = 'Ex3_Flims/'
for i in flims_name:
    ffprobe = FFprobe(file_loc+i)
    print(ffprobe. checkFile())
    #print(ffprobe. convertFile())
    print("="*70)

Report for file Ex3_Flims/Cosmos_War_of_the_Planets.mp4
-- Wrong frame rate. Expected 25 FPS, found 29.97002997002997 FPS 
-- Wrong aspect ratio. Expected 16:9, found 314:177
-- Wrong resolution. Expected 640x360, found 628x354
-- Wrong audio bit rate. Expected up to 256 kb/s, found 317.103 kb/s

Report for file Ex3_Flims/Last_man_on_earth_1964.mov
-- Wrong video container format. Expected mp4, found .mov 
-- Wrong video codec. Expected h.264, found prores
-- Wrong frame rate. Expected 25 FPS, found 23.976023976023978 FPS 
-- Wrong video bit rate. Expected 2 - 5 Mb/s, found 9.285191 Mb/s
-- Wrong audio bit rate. Expected up to 256 kb/s, found 1536.0 kb/s
-- Wrong audio codec. Expected aac, found pcm_s16le

Report for file Ex3_Flims/The_Gun_and_the_Pulpit.avi
-- Wrong video container format. Expected mp4, found .avi 
-- Wrong video codec. Expected h.264, found rawvideo
-- Wrong aspect ratio. Expected 16:9, found 0:1
-- Wrong resolution. Expected 640x360, found 720x404
-- Wrong video bit

## Check the converted file format

In [5]:
flims_name_ok = ["Cosmos_War_of_the_Planets_formatOk.mp4","Last_man_on_earth_1964_formatOk.mp4","The_Gun_and_the_Pulpit_formatOk.mp4",
    "The_Hill_Gang_Rides_Again_formatOk.mp4","Voyage_to_the_Planet_of_Prehistoric_Women_formatOk.mp4"]
file_loc = 'Ex3_Flims/'
for i in flims_name_ok:
    ffprobe = FFprobe(file_loc+i)
    print(ffprobe. checkFile())
    #print(ffprobe. convertFile())
    print("="*70)

Report for file Ex3_Flims/Cosmos_War_of_the_Planets_formatOk.mp4
-- All formatting are okay
Report for file Ex3_Flims/Last_man_on_earth_1964_formatOk.mp4
-- All formatting are okay
Report for file Ex3_Flims/The_Gun_and_the_Pulpit_formatOk.mp4
-- All formatting are okay
Report for file Ex3_Flims/The_Hill_Gang_Rides_Again_formatOk.mp4
-- All formatting are okay
Report for file Ex3_Flims/Voyage_to_the_Planet_of_Prehistoric_Women_formatOk.mp4
-- All formatting are okay
